In [1]:
# Импортируем нужные библиотеки
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn import metrics
from sklearn.impute import SimpleImputer

# Загрузка датасета
file_path = 'Arrest_Data_from_2010.csv'  

# Загружаем датасет
dataset = pd.read_csv(file_path)                                 


#### Загружаем датасет, содержащий информацию о данных арестов. Целевой признак для классификации — это код типа ареста (`Arrest Type Code`).

#### Подготовка данных

Выделяем целевой признак и признаки для модели. Удаляем текстовые данные и заполняем пропущенные значения.

In [2]:
# Используем только 5% данных из датасета
dataset_sampled = dataset.sample(frac=0.05, random_state=42)

In [3]:
# Предобработка данных
y = dataset_sampled['Arrest Type Code']
# Предположим, что 'M' — целевой класс, а все остальные — в "другие"
y = y.apply(lambda x: 1 if x == 'M' else 0)

X = dataset_sampled.drop('Arrest Type Code', axis=1)

# Удаляем столбцы с текстовыми данными
X = X.select_dtypes(exclude=['object'])

# Заполнение пропущенных значений
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)


#### Нормализация данных

Нормализуем данные для улучшения работы модели.

In [4]:
# Нормализация данных
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X_imputed)


#### Разделение данных на тренировочную и тестовую выборки

Разделяем данные на тренировочную и тестовую выборки.

In [5]:
# Разделение на тренировочную и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


#### Поиск лучших гиперпараметров

Используем RandomizedSearchCV для нахождения лучших гиперпараметров модели K-ближайших соседей.

In [6]:
# Поиск гиперпараметров с помощью RandomizedSearchCV
kn_classifier = KNeighborsClassifier()
kn_params = {
    'n_neighbors': range(1, 10),  # Уменьшенный диапазон
    'metric': ['euclidean', 'manhattan']  # Меньше метрик
}

kn_grid = RandomizedSearchCV(kn_classifier, kn_params, n_iter=10, cv=3, n_jobs=-1)
kn_grid.fit(X_train, y_train)

# Лучшие параметры модели
kn_best_params = kn_grid.best_params_
print('Лучшие параметры:', kn_best_params)

Лучшие параметры: {'n_neighbors': 5, 'metric': 'manhattan'}


#### Обучение и оценка модели

Обучаем модель с найденными оптимальными гиперпараметрами и оцениваем её производительность.

In [7]:
# Обучение модели с лучшими параметрами
kn_best_model = KNeighborsClassifier(**kn_best_params)
kn_best_model.fit(X_train, y_train)

# Прогнозирование
kn_predicted = kn_best_model.predict(X_test)

# Оценка модели
print('Оценка модели:\n', metrics.classification_report(y_test, kn_predicted, digits=5))

Оценка модели:
               precision    recall  f1-score   support

           0    0.73620   0.71959   0.72780      5228
           1    0.81901   0.83112   0.82502      7982

    accuracy                        0.78698     13210
   macro avg    0.77761   0.77535   0.77641     13210
weighted avg    0.78624   0.78698   0.78655     13210

